In [29]:
import sys
sys.path.append("../.")

In [30]:
import pandas as pd
from Utils.Medic1Utils import checkDateTimeFormat, divideInPosts, isPostFromDoctor, stripAndJoin

In [31]:
df = pd.read_pickle("MedicItaliaDataNotClean.pkl")

In [32]:
element: str = df.iat[2000,0]
elementSplitted = element.split("\n")

In [33]:
add = True
entireText = []
for line in elementSplitted:
    if "Questo consulto ha ricevuto" in line:
        add = False
    if add:
        if "👉🏻" not in line and "Nutricia Fortimel" not in line:
            entireText.append(line)

In [34]:
domandaSplitted = []
indexesToRemove = []

for i,line in enumerate(entireText):
    if checkDateTimeFormat(line):
        break
    domandaSplitted.append(line)
    indexesToRemove.append(i)
    
textSenzaDomanda = []
for i in range(len(entireText)):
    if i not in indexesToRemove:
        textSenzaDomanda.append(entireText[i])

textSenzaDomanda = textSenzaDomanda[1::]

domanda = "\n".join(domandaSplitted)

In [35]:
posts = divideInPosts(textSenzaDomanda, "[#")

print("###UTENTE###")
print(domanda)

for post in posts:
    print()
    if isPostFromDoctor(post):
        print("###DOTTORE###")
        print(stripAndJoin(post[4::]))
    else:
        print("###UTENTE###")
        print(stripAndJoin(post[2::]))
    

###UTENTE###
Aiuto problemi intestinali :(
Supplementi nutrizionali per pazienti oncologici: come alimentarsi nel modo giusto 
scrivo qui nel forum perchè non so più a chi rivolgermi...
vi racconto il mio problema, tutto è iniziato verso dicembre,
avevo dolori addominali e diarre (più che diarrea feci molle e deforme)
il tutto iniziava dopo i pasti, accusavo un senso di malessere in tutta la zona addominale che poi uniti all'ansia mi provocava giramenti di testa, nausea e una forte urgenza di andare al bagno...dopo la defecazione stavo leggermente meglio.
All'inizio pensavo fosse qualcosa che ho mangiato o un colpo di freddo o un virus, ma è andato a avanti per circa 2 mesi e peggiorava.
sono andato dal medico di famiglia e senza visitarmi mi prescrisse 2 antibiotici e delle punture di cortisone perchè secondo lui avevo la colite ulcerosa e sindrome del colon irritabile...premesso che non avevo sangue nelle feci..
inizio la cura con l'adeguata alimentazione ma le evaquazioni erano più 